<a href="https://colab.research.google.com/github/eman-kom/tensorflow-fundamentals/blob/main/05_intro_to_computer_vision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>